### `tf.function` allows to switch from eager execution to graph execution

eager execution means TensorFlow operations are executed in Python, and results are returned to Python unlike in `Graph Execution` where tensor computations are executed as `tf.Graph`. It contains set of `tf.Operation` and `tf.Tensor` objects.

They're useful in environments that do not have Python interpreter, like mobile applications. They are easily optimized, and let TensorFlow run faster and also in parallel.

In [1]:
import tensorflow as tf
import timeit
from datetime import datetime

2023-04-04 14:58:57.943432: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-04 14:58:58.070382: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-04 14:58:58.094501: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-04 14:58:58.559497: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

A `tf.Function` can be called directly as a function or a decorator. It takes a python function as an input and returns a function, which makes TensorFlow graph from python functions.

In [2]:
def function(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

graph_function = tf.function(function)

x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = function(x1, y1, b1).numpy()
tf_value = graph_function(x1, y1, b1).numpy()

2023-04-04 14:59:00.333005: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-04-04 14:59:00.333058: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ssk-Dell-G15-5511
2023-04-04 14:59:00.333062: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ssk-Dell-G15-5511
2023-04-04 14:59:00.333180: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.85.12
2023-04-04 14:59:00.333195: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.85.12
2023-04-04 14:59:00.333198: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 525.85.12
2023-04-04 14:59:00.334088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in pe

 Another way to create or call a tf.Function is by using it as a decorator

In [6]:
def inner_function(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

# Using the decorator to call
@tf.function
def outer_function (x):
    y = tf.constant([[1.0], [4.0]])
    b = tf.constant(5.0)
    return inner_function(x, y, b)

# This will now create a graph that includes the inner function as well as the outer function
outer_function(tf.constant([[1.0, 3.0]])).numpy()

array([[18.]], dtype=float32)

TensorFlow code will have TF operations as well as Python logic, so we need to undergo the extra step to add it to the graph. `tf.function` uses a library `tf.autograph` to convert Python code to graph

In [11]:
def simple_relu(x):
    if tf.greater(x, 0):
        return x
    else:
        return 0
    
print("Original Python function: ", simple_relu(tf.constant(1)).numpy())

# This will create a graph that includes the if statement
tf_simple_relu = tf.function(simple_relu)

print("Function converted to Tensorflow: ", tf_simple_relu(tf.constant(1)).numpy())

Original Python function:  1
Function converted to Tensorflow:  1


In [14]:
# This is the graph code
print(tf.autograph.to_code(simple_relu))

def tf__simple_relu(x):
    with ag__.FunctionScope('simple_relu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal retval_, do_return
            (do_return, retval_) = vars_

        def if_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = ag__.ld(x)
            except:
                do_return = False
                raise

        def else_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = 0
            except:
                do_return = False
                raise
        ag__.if_stmt(ag__.converted_call(ag__.ld(tf).greater, (ag__.ld(x), 0), None, fscope), if_bo

In [16]:
# This is the graph
print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "Greater/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 0
      }
    }
  }
}
node {
  name: "Greater"
  op: "Greater"
  input: "x"
  input: "Greater/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "cond"
  op: "StatelessIf"
  input: "Greater"
  input: "x"
  attr {
    key: "Tcond"
    value {
      type: DT_BOOL
    }
  }
  attr {
    key: "Tin"
    value {
      list {
        type: DT_INT32
      }
    }
  }
  attr {
    key: "Tout"
    value {
      list {
        type: DT_BOOL
        type: DT_INT32
      }
    }
  

Every time a new graph with a set of arguments that involve a new dtype, they can't be handled by existing graphs, so the function creates a new tf.Graph specialized to those arguments

The type specification of a graph's inputs are known as the its `input signature` or just `signature`. The function store the tf.Graph corresponding to that signature in a `ConcreteFunction` wrapper

In [27]:
@tf.function
def tf_relu(x):
    return tf.maximum(0., x)

''' Since graphs are created only when funtions are called,
    the below calls will create new graphs for each call '''

print("This call creates graph with input signature of float: ", tf_relu(tf.constant(3.3)))
print("This call creates graph with input signature of array: ", tf_relu([1, -1]))
print("This call creates graph with input signature of float array: ", tf_relu(tf.constant([1.0, -1.0])))

This call creates graph with input signature of float:  tf.Tensor(3.3, shape=(), dtype=float32)
This call creates graph with input signature of array:  tf.Tensor([1. 0.], shape=(2,), dtype=float32)
This call creates graph with input signature of float array:  tf.Tensor([1. 0.], shape=(2,), dtype=float32)


In [28]:
# The following calls will not create new graphs as the signatures have been called before
print("This signature has been used before: ", tf_relu(tf.constant(2.2)))
print("This signature has been used before: ", tf_relu(tf.constant([-2., 2.])))

This signature has been used before:  tf.Tensor(2.2, shape=(), dtype=float32)
This signature has been used before:  tf.Tensor([0. 2.], shape=(2,), dtype=float32)


In [29]:
print(tf_relu.pretty_printed_concrete_signatures())

tf_relu(x)
  Args:
    x: float32 Tensor, shape=()
  Returns:
    float32 Tensor, shape=()

tf_relu(x=[1, -1])
  Returns:
    float32 Tensor, shape=(2,)

tf_relu(x)
  Args:
    x: float32 Tensor, shape=(2,)
  Returns:
    float32 Tensor, shape=(2,)


You can turn off function's ability to create and run graphs. We can make it execute eagerly using `tf.config.run_functions_eagerly(True)`

In [39]:
tf.config.run_functions_eagerly(True)
tf_relu(tf.constant(1.0))

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [38]:
tf.config.run_functions_eagerly(False)
tf_relu(tf.constant(1.0))

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

# Graph vs Eager execution

In [59]:
@tf.function
def get_MSE(y_true, y_pred):
    print("Calculating MSE")
    return tf.reduce_mean(tf.square(y_true - y_pred))

In [60]:
y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)

print(y_true)
print(y_pred)

tf.Tensor([7 4 0 1 0], shape=(5,), dtype=int32)
tf.Tensor([7 0 2 0 2], shape=(5,), dtype=int32)


In [61]:
# Graph execution
tf.config.run_functions_eagerly(False)

get_MSE(y_true, y_pred)
get_MSE(y_true, y_pred)
get_MSE(y_true, y_pred)

Calculating MSE


<tf.Tensor: shape=(), dtype=int32, numpy=5>

In [62]:
# Eager execution
tf.config.run_functions_eagerly(True)

get_MSE(y_true, y_pred)
get_MSE(y_true, y_pred)
get_MSE(y_true, y_pred)

Calculating MSE
Calculating MSE
Calculating MSE


<tf.Tensor: shape=(), dtype=int32, numpy=5>

As you can see, the print statement executes only once in graph execution. This is because the function runs original code to create graph by tracing which only captures TensorFlow operations into a graph. So the Python code is `run` only once and not captured in the graph so it is never executed again.

We can overcome this using the TensorFlow version of the functions like `tf.print` for the above case

In [63]:
@tf.function
def get_MSE(y_true, y_pred):
    tf.print("Calculating MSE")
    return tf.reduce_mean(tf.square(y_true - y_pred))

In [64]:
# Graph execution
tf.config.run_functions_eagerly(False)

get_MSE(y_true, y_pred)
get_MSE(y_true, y_pred)
get_MSE(y_true, y_pred)

Calculating MSE
Calculating MSE
Calculating MSE


<tf.Tensor: shape=(), dtype=int32, numpy=5>

## Speed difference between graph and eager execution

In [65]:
x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)

def power(x, y):
  result = tf.eye(10, dtype=tf.dtypes.int32)
  for _ in range(y):
    result = tf.matmul(x, result)
  return result

In [66]:
print("Eager execution:", timeit.timeit(lambda: power(x, 100), number=1000), "seconds")

power_graph = tf.function(power)

print("Graph execution:", timeit.timeit(lambda: power_graph(x, 100), number=1000), "seconds")

Eager execution: 2.2354137409984105 seconds
Graph execution: 0.3370804419992055 seconds


# Non-strict execution

Graph execution only executes the operations necessary to produce observable results like the return function.
Side effects:-
    1. Input/Output operations
    2. Debugging operations like assert in `tf.debugging`
    3. Mutations of tf.Variable

### Graphs speed up code majorly, but takes a lot of time to trace a function, that is, create the graph. However this tradeoff is worth as you only trace once.

Use `print` statements within functions to know whenever your graph traces. They execute only once, which the first time when the function traces, so we will come to know when a function is being traced.